# Evaluation of Images of Lines from Writer

1. Set the DATA_FOLDER
2. Set the OUTPUT_FILE_NAME
3. Set the CUDA properties
4. Run the jupyter cells.

Output is in a file called OUTPUT_FILE_NAME in the DATA_FOLDER

In [1]:
DATA_FOLDER= '/data/iam_data/words/'
OUTPUT_FILE_NAME='words_baseline.csv'

In [2]:
with open('/data/iam_data/ascii/forms.txt','r') as fp:
    lines = [l for l in fp.readlines() if l[0] != '#']
    writer_id = {l[0]:l[1] for l in [x.split(' ') for x in lines]}
    
with open('/data/iam_data/ascii/words.txt','r') as fp:
    lines = [l for l in fp.readlines() if l[0] != '#']
    word_id = {l[0]:l[-1].strip() for l in [x.split(' ') for x in lines]}

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="1"; 

In [4]:
from __future__ import division
import numpy as np
import os
from keras.utils import to_categorical, np_utils
import pandas as pd

/home/robertsone/HWR/writer_identification/wi/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [5]:
from model import build_model
num_classes = 50
model = build_model(num_classes)
model.load_weights('writer_id_checkpoint.hdf5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 115, 115, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 58, 58, 32)        832       
_________________________________________________________________
activation (Activation)      (None, 58, 58, 32)        0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 29, 29, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 29, 29, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 29, 29, 64)        0         
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 14, 14, 64)        0

In [6]:
from keras.models import Model
model2= Model(inputs=model.input, outputs=model.layers[-5].output)
model2.summary()
model = None

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_input (InputL [(None, 113, 113, 1)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 115, 115, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 58, 58, 32)        832       
_________________________________________________________________
activation (Activation)      (None, 58, 58, 32)        0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 29, 29, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 29, 29, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 29, 29, 64)       

In [63]:
from PIL import Image
from random import *
import numpy as np

def local_prep_sample(image_file_name, slide=55):
    """
    Get a list of slices for the provide image:
    generates random crops from each line
    """
    im = Image.open(image_file_name)
    im = im.convert('L')
    cur_width = im.size[0]
    cur_height = im.size[1]

                # print(cur_width, cur_height)
    height_fac = 113 / cur_height

    new_width = int(cur_width * height_fac)
    size = new_width, 113

    imresize = im.resize((size), Image.ANTIALIAS)  # Resize so height = 113 while keeping aspect ratio
    now_width = imresize.size[0]
    now_height = imresize.size[1]
                # Generate crops of size 113x113 from this resized image and keep random 10% of crops

    now_width - 113
    pick_num = int(now_width/slide) + 1

    im = np.asarray(imresize)
    images = None
    for start_i in range(pick_num):
            start = start_i * slide
            if images is None:
                if start+113 <= im.shape[1]:
                    images = im[:, start:start+113].reshape((1,113,113))
            else:
                #print (images.shape)
                if start+113 > im.shape[1]:
                    start -= (113+start-im.shape[1])
                im_s = im[:, start:start+113].reshape((1,113,113))
                #print (im_s.shape)
                images = np.vstack((images,im_s))
    return images

In [64]:


# get slices of normalized image
def generate_predict_data(filename):
    image_arrays = local_prep_sample(filename)
    if image_arrays is not None:
        #reshape image_arrays for feeding in later
        image_arrays = image_arrays.reshape(image_arrays.shape[0], 113, 113, 1)
        #convert to float and normalize
        image_arrays = image_arrays.astype('float32')
        image_arrays /= 255

    return image_arrays

In [ ]:
from scipy import stats
    
folders = [DATA_FOLDER]
for folder in folders:
    with open(os.path.join(folder,OUTPUT_FILE_NAME),'w') as fp:
        fp.write('filename,word,writer,'  + ','.join([str(i) for i in range(256)])  + '\n')
        print(folder)
        for root, dirs, files in os.walk(folder, topdown=False):
            for file in files:
                if file.endswith('png') or file.endswith('jpg'):
                    images = generate_predict_data(os.path.join(root, file))
                    if images is not None:
                        predictions = model2.predict(images)
                        ostr = ','.join([str(x) for x in np.mean(predictions, axis=0)])
                        sub = os.path.splitext(file)[0]
                        word = word_id[sub]
                        writer = writer_id['-'.join(sub.split('-')[:2])]
                        fp.write(f'{file},{word},{writer},{ostr}\n')

/data/iam_data/words/


In [57]:
generate_predict_data('/data/iam_data/words/a01/a01-000u/a01-000u-00-01.png').shape

(10, 113, 113, 1)